In [1]:
import sqlite3

In [21]:
# connection objects represent the database, to access a database first construct a connection

conn = sqlite3.Connection('example.db')
conn.row_factory = sqlite3.Row

In [22]:
#  database cursor - used to manage the context of a fetch operation

c = conn.cursor()

In [3]:
# execute an SQL statement

c.execute('''CREATE TABLE employees (first_name text, second_name text, ID int)''')
c.execute('''INSERT INTO employees VALUES ('Mahmud', 'Siraj', 1)''')

# save the changes
conn.commit()

In [4]:
# to prevent SQL injection use DB-API parameter substitution
# execute many will execute over entire sequence of parameters provided

t = [('Liva', 'Sikder', 2), ('Shaheda', 'Siraj', 3), ('Cristiano', 'Ronaldo', 4)]

c.executemany('''INSERT INTO employees VALUES (?,?,?)''', t)

In [15]:
# To retrieve data after executing a SELECT statement:
#
#    1. Treat the cursor as an iterator 
#    2. Call the cursor’s fetchone() method to retrieve a single matching row
#    3. Call fetchall() to get a list of the matching rows.


for row in c.execute('''SELECT * FROM employees'''):
    print(row)

('Mahmud', 'Siraj', 1)
('Liva', 'Sikder', 2)
('Liva', 'Sikder', 2)
('Shaheda', 'Siraj', 3)
('Cristiano', 'Ronaldo', 4)


In [6]:
# Create a custom aggregate function

class MySum:
    def __init__(self):
        self.count = 0
        
    def step(self, value): # method parameters are passed to
        self.count += value
        
    def finalize(self): # method returning the result of the SQL statement
        return self.count

conn.create_aggregate("MYSUM",1,MySum)
c.execute('''SELECT MYSUM(ID) FROM employees''')
print(c.fetchone()[0])

12


In [18]:
# dump the database in SQL text format 

with open("dump.sql", 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' % line)

In [37]:
c.close()
conn.commit()
conn.close()

In [35]:
# create a custom data type

class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y
        
    def __repr__(self):
        return "(%f;%f)" % (self.x, self.y)

def adapt_point(point):
    return ("%f;%f" % (point.x, point.y)).encode('ascii')

def convert_point(s):
    x, y = list(map(float, s.split(b";")))
    return Point(x, y)

# Register the adapter
sqlite3.register_adapter(Point, adapt_point)

# Register the converter
sqlite3.register_converter("point", convert_point)

p = Point(4.0, -3.2)

In [38]:
#########################
# 1) Using declared types
con = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()
cur.execute("create table test(p point)")

cur.execute("insert into test(p) values (?)", (p,))
cur.execute("select p from test")
print("with declared types:", cur.fetchone()[0])
cur.close()
con.close()

with declared types: (4.000000;-3.200000)


In [39]:
#######################
# 2) Using column names
con = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_COLNAMES)
cur = con.cursor()
cur.execute("create table test(p)")

cur.execute("insert into test(p) values (?)", (p,))
cur.execute('select p as "p [point]" from test')
print("with column names:", cur.fetchone()[0])
cur.close()
con.close()

with column names: (4.000000;-3.200000)
